## Import dependencies

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as db
import config
import requests
import csv
import googlemaps
import datetime

from pathlib import Path
from dateutil.parser import parse
from datetime import date, datetime
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, Integer, Table, Column, MetaData

## Set common variables

In [2]:
# Create SQLite db
restaurants_db = "restaurants.sqlite"
connector = (f"sqlite:///{restaurants_db}")

# Create engine
engine = create_engine(connector)

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes (tables) mapped to the Base
Base.classes.keys()

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Dbg flag indicates if a debug execution or not, and will conditionally skip certain tasks both in this cell and within the geocode function
# This is so that the code can be executed in "demo" mode and will reduce Google API calls to avoid charges
dbg = "Y"

# Set API key from config file
gmaps_key = googlemaps.Client(key=config.API_KEY)

## Create helper functions

In [3]:
def verify_inserts(tablename):
    # Using the inspector to print the column names within the 'restaurants' table and its types
    columns = inspector.get_columns(tablename)
    for column in columns:
        print(column["name"], column["type"])

    # Create metadata access to the object
    meta_data = db.MetaData(bind=engine)
    db.MetaData.reflect(meta_data)

    # Get the table from the metadata object
    table = meta_data.tables[tablename]

    # Select and print rowcount
    result = db.select([db.func.count()]).select_from(table).scalar()
    print("Row Count:", result)
    
    table_df = pd.read_sql_table(
        tablename,
        con=engine
    )

    table_df = table_df.head(10)
    
    return table_df

In [4]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

In [5]:
def str_to_date(date_string):
    date = datetime.strptime(date_string, '%Y-%m-%d')
    return date

In [6]:
def add_change_date(df):
    df["change_date"] = date.today()
    return df

In [7]:
def str_to_bool(df, col):
    return col.map({'True': True, 'yes': True, 1: True, 0: False, 'False': False, 'Na':False, 'no':False,})

In [8]:
def substr_filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

In [9]:
def reduce_dims(search_string, df):
    print("Reducing columns like " + search_string)
    
    # Create a dataframe as a lookup table 
    type_col_df = pd.DataFrame((df.filter(regex=search_string).columns))

    # Create a description column for mapping
    type_col_df["description"] = type_col_df[0].str.replace(search_string,"")
    type_col_df["description"] = type_col_df[0].str.replace("_","")

    # Create an identity column
    type_col_df['id'] = type_col_df.index

    # Reorder columns so key is in the first position
    type_col_df.insert(0, 'id', type_col_df.pop('id'))

    # Increment by 1 so remaining "false" values can be set to 0 as a placeholder
    type_col_df["id"] = type_col_df["id"] + 1

    # Rename column 0
    type_col_df.rename(columns = {0:'column_name'}, inplace = True)

    # Derive new column names
    new_attr_id = search_string + "TypeId"
    new_attr_type = search_string + "Type"
    
    # for each unique column matching the search string type
    for index, row in type_col_df.iterrows():
        # set a new column with the value of the id from the type columns dataframe
        df.loc[df[row['column_name']] == True, new_attr_id] = row["id"]
        df[new_attr_id] = df[new_attr_id].fillna(0).astype("int")

        # set a new column with the value of the description from the type columns dataframe
        df.loc[df[row['column_name']] == True, new_attr_type] = row["description"]
        df[new_attr_type] = df[new_attr_type].fillna("na")

    # Drop reduced columns
    df = df.drop(type_col_df["column_name"], axis=1)
    
    type_col_df = type_col_df.drop("column_name", axis=1)

    # Write lookup table to database
    type_col_df.to_sql(new_attr_type, engine, if_exists='replace')
    
    return df

In [10]:
def encode_cols (col_name, df):
    print("Encoding column " + col_name)
    
    # Create a dataframe as a lookup table 
    type_col_df = pd.DataFrame(df[col_name].unique())

    # Create an identity column
    type_col_df['id'] = type_col_df.index

    # Reorder columns so key is in the first position
    type_col_df.insert(0, 'id', type_col_df.pop('id'))

    # Increment by 1 so remaining "false" values can be set to 0 as a placeholder
    type_col_df["id"] = type_col_df["id"] + 1

    # Rename column 0
    type_col_df.rename(columns = {0:'description'}, inplace = True)

    new_attr_id = col_name + "Id"
    new_attr_type = col_name + "Type"
    
    # Write lookup table to database
    type_col_df.to_sql(new_attr_type, engine, if_exists='replace')

    # for each unique column matching the search string type
    for index, row in type_col_df.iterrows():
        # set the column to the value of the id from the type columns dataframe
        df.loc[df[col_name] == row["description"], new_attr_id] = row["id"]
        df[new_attr_id] = df[new_attr_id].fillna(0).astype("int")
        
    return df

## Clean and load business data for mexican_restaurants.csv

In [11]:
def etl_mex_restaurants(df, gmaps_key, dbg):
    # Limit to Mexican restaurants only as that is our dashboard focus
    # Saves about 12,000 API calls and probably a few dineros!

    # Create a formatted address string for geocoding
    df['full_address'] = df[['address', 'city', "state",'postal_code']].agg(', '.join, axis=1).str.replace('"','')

    # Run the address through the geocoder function and apply the results to a new column called "geocoded"
    df['geocoded'] = df['full_address'].apply(geocode)

    # # Apply the new geocodes to the restaurants dataframe
    df[['latitude', 'longitude']] = pd.DataFrame(df['geocoded'].tolist(), index=df.index)

    # For "normal" mode only, debug mode limits dataset to cut down on API calls
    if dbg == "N": 
        df.to_csv("mexican_restaurants.csv")
        
    return df.head(100)

## Load and clean yelp_business.csv

In [12]:
def etl_restaurants(dbg, api_key):
    # Load business into df
    business_path = Path('../data_zipped/yelp_business.csv.zip', low_memory=False)
    business_df = pd.read_csv(business_path)

    # Filter businesses to restaurants only
    business_df = business_df[business_df["categories"].str.contains("Restaurant")]
    
    # Drop rows where is_open = 0
    business_df = business_df[business_df.is_open == 0]

    # Add flag "is_mexican_restaurant"
    business_df["is_mexican_restaurant"] = np.where(business_df.categories.str.contains("Mex"),True,False).astype(bool)
    
    # Remove is_open, neighborhood, and categories
    business_df = business_df.drop(['is_open', 'neighborhood', 'categories'], axis=1)

    # Fill missing values with empty string
    business_df = business_df.fillna('')

    # Add change date
    add_change_date(business_df)
    
    # Filter input df to only geocode Mexican Restaurants to save on API calls
    mex_restaurants_df = business_df[business_df["is_mexican_restaurant"] == 1]

    # For "debug" mode only, reduces API calls
    if dbg == 'Y':
        mex_restaurants_df = mex_restaurants_df.head()

    etl_mex_restaurants(mex_restaurants_df, gmaps_key, dbg)
    
    # Write to SQLite db
    business_df.to_sql('Restaurants', engine, if_exists='replace')
    
    return business_df


## Load and clean Reviews

In [13]:
def etl_reviews(df):
# Load comments into df
    reviews_path = Path('../data_zipped/yelp_tip.csv.zip', low_memory=False)
    reviews_df = pd.read_csv(reviews_path)

    # Drop rows with missing values
    reviews_df = reviews_df.dropna(axis=0,how="any")

    # Convert "date" column to a date datatype
    reviews_df['date'] = reviews_df['date'].apply(str_to_date)
    
    # Merge with business_df to filter data set to relavent rows
    reviews_df = reviews_df.set_index('business_id').join(business_df.set_index('business_id'), rsuffix='_review')

    # Eliminate unneeded columns
    reviews_df = reviews_df.loc[:,"text":"user_id"]
    
    # Name dataframe index and write to table
    reviews_df['review_id'] = reviews_df.index
    
    # Reorder columns so key is in the first position
    reviews_df.insert(0, 'review_id', reviews_df.pop('review_id'))
    
    # Add ChangeDate
    add_change_date(reviews_df)
    
    # Send entire reviews_df to csv for BI
    reviews_df.to_csv('../data_zipped/restaurants.csv')
    
    # Write to database
    reviews_df.to_sql('Reviews', engine, if_exists='replace')

    # Release memory
    del reviews_df

## Load and clean yelp_business_attributes.csv

In [14]:
def etl_restaurant_attributes(df):
    # Load business attributes into df
    attributes_path = Path('../data_zipped/yelp_business_attributes.csv.zip', low_memory=False)
    attributes_df = pd.read_csv(attributes_path)

    # Rearrange column names (not columns) due to defect in source data set
    col_list = list(attributes_df)
    col_list.insert(2, col_list.pop())
    attributes_df.columns = col_list

    # Merge with business_df to filter data set to relavent rows
    restaurant_attribute_df = business_df.set_index('business_id').join(attributes_df.set_index('business_id'), rsuffix='_attr')

    # Release memory
    del attributes_df

    # List of category columns
        # Column: Alcohol, Values: ['Na' 'full_bar' 'beer_and_wine' 'none']
        # Column: NoiseLevel, Values: ['Na' 'average' 'very_loud' 'quiet' 'loud']
        # Column: RestaurantsAttire, Values: ['Na' 'casual']
        # Column: WiFi, Values: ['Na' 'free' 'paid' 'no']
        # Column: Smoking, Values: ['Na' 'no' 'outdoor' 'yes']
        # Column: BYOBCorkage, Values: ['Na' 'yes_free' 'no' 'yes_corkage']
    string_cols = ["Alcohol","NoiseLevel","WiFi","Smoking","BYOBCorkage"]

    # Convert 'Na' to 0 and then convert col to int
    # Column: RestaurantsPriceRange2, Values: ['Na' '1' '3' '2' '4']
    restaurant_attribute_df["RestaurantsPriceRange2"] = restaurant_attribute_df["RestaurantsPriceRange2"].replace(to_replace='Na', value=0)

    # Convert remaining "Na" to "no", affects these columns:
    restaurant_attribute_df[string_cols] = restaurant_attribute_df[string_cols].replace(to_replace='Na', value='no')

    # For column in the dataframe:
    for col in col_list[1:]:
        # Skip category columns for string conversion
        if col not in string_cols:
            # Skip columns already typed boolean 
            if col not in list(restaurant_attribute_df.select_dtypes(include=['bool']).columns):
                # Convert to boolean
                if(restaurant_attribute_df.loc[:,col].nunique()) <= 3:
                    restaurant_attribute_df[col] = str_to_bool(restaurant_attribute_df,restaurant_attribute_df[col])

        # Eliminate columns with only one distinct value
        if(restaurant_attribute_df.loc[:,col].nunique()) == 1:
            restaurant_attribute_df.pop(col)

    # Drop rows without attributes
    restaurant_attribute_df = restaurant_attribute_df.dropna(axis=0,how="any")
    
    ## Dimensionality Reduction
    # Collapse into categorical attributes:
        # BusinessParkingType:
            # BusinessParking_garage                 2
            # BusinessParking_street                 2
            # BusinessParking_validated              2
            # BusinessParking_lot                    2
            # BusinessParking_valet                  2
        # MusicType:
            # Music_dj                               2
            # Music_karaoke                          2        
        # Restaurants:
            # RestaurantsReservations                2
            # RestaurantsTakeOut                     2
            # RestaurantsDelivery                    2
            # RestaurantsGoodForGroups               2
        # GoodForMealType:
            # GoodForMeal_dessert                    2
            # GoodForMeal_latenight                  2
            # GoodForMeal_lunch                      2
            # GoodForMeal_dinner                     2
            # GoodForMeal_breakfast                  2
            # GoodForMeal_brunch                     2
        # DietaryRestrictionsType:
            # DietaryRestrictions_dairy-free         2
            # DietaryRestrictions_gluten-free        2
            # DietaryRestrictions_vegan              2
            # DietaryRestrictions_kosher             2
            # DietaryRestrictions_halal              2
            # DietaryRestrictions_soy-free           2
            # DietaryRestrictions_vegetarian         2
        # BestNightsType:
            # BestNights_monday                      2
            # BestNights_friday                      2
            # BestNights_wednesday                   2
            # BestNights_thursday                    2
            # BestNights_sunday                      2
            # BestNights_saturday                    2

    search_strings = ["BusinessParking","Music","Restaurants","GoodForMeal","DietaryRestrictions","BestNights"]
    for str in search_strings:
        restaurant_attribute_df = reduce_dims(str, restaurant_attribute_df)

    # Encode the remaining string columns
    for col in string_cols:
        restaurant_attribute_df = encode_cols(col, restaurant_attribute_df)
        
    # Send entire restaurant_attribute_df to csv for BI
    restaurant_attribute_df.to_csv('../data_zipped/restaurants.csv')

    # Eliminate business_df elements
    attribute_df = restaurant_attribute_df.loc[:,"change_date":]
    
    # Release memory
    del restaurant_attribute_df

    # Write to SQLite db
    attribute_df.to_sql('RestaurantAttributes', engine, if_exists='replace')
    
    # Release memory
    del attribute_df

In [15]:
# Run Restaurants ETL
business_df = etl_restaurants(dbg,gmaps_key)

# Validate success
verify_inserts('Restaurants')

index BIGINT
business_id TEXT
name TEXT
address TEXT
city TEXT
state TEXT
postal_code TEXT
latitude FLOAT
longitude FLOAT
stars FLOAT
review_count BIGINT
is_mexican_restaurant BOOLEAN
change_date DATE
Row Count: 14225


,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_mexican_restaurant,change_date
0,10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""","""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,3,False,2022-08-12
1,15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""","""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,12,False,2022-08-12
2,29,gAy4LYpsScrj8POnCW6btQ,"""Toast Cafe""","""2429 Hwy 160 W""",Fort Mill,SC,29708,35.047287,-80.990559,3.5,6,False,2022-08-12
3,32,1_3nOM7s9WqnJWTNu2-i8Q,"""Le Bistro Balmoral""","""305 Rue Sainte-Catherine O""",Montreal,QC,H2X 2A1,45.506772,-73.566725,3.0,8,False,2022-08-12
4,44,BnuzcebyB1AfxH0kjNWqSg,"""Carrabba's Italian Grill""","""245 Lancaster Ave""",Frazer,PA,19355,40.041003,-75.542497,3.5,25,False,2022-08-12
5,64,EJFdWX908N8Yc2XG0Lky8A,"""River Moon Cafe""","""104 43rd St""",Pittsburgh,PA,15201,40.472735,-79.963265,4.0,5,False,2022-08-12
6,91,F0fEKpTk7gAmuSFI0KW1eQ,"""Cafe Mastrioni""","""4250 S Rainbow Blvd, Ste 1007""",Las Vegas,NV,89103,36.111057,-115.241688,1.5,3,False,2022-08-12
7,97,HAX1zec191t7QkT2sBZ76A,"""La Isla Cuban Restaurant""","""1816 Galerea Blvd, Ste D""",Charlotte,NC,28270,35.137223,-80.734594,3.0,4,False,2022-08-12
8,104,1nhf9BPXOBFBkbRkpsFaxA,"""Mirage Grill & Lounge""","""117 Eglinton Avenue E""",Toronto,ON,M4P 1H4,43.707465,-79.394285,2.0,6,False,2022-08-12
9,118,T5CdfrZWw-uW9Y5L_sddqQ,"""Police Station Pizza""","""7235 Steubenville Pike""",Oakdale,PA,15071,40.442828,-80.186293,3.0,9,False,2022-08-12


In [16]:
# Run Reviews ETL
etl_reviews(business_df)

# Validate success
verify_inserts('Reviews')

business_id TEXT
review_id TEXT
text TEXT
date DATETIME
likes BIGINT
user_id TEXT
change_date DATE
Row Count: 1098322


,business_id,review_id,text,date,likes,user_id,change_date
0,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,"Combo A: Roast duck, roast pork, Singapore noo...",2015-10-12,0,6tbXpUIU6upoeqWNDo9k_A,2022-08-12
1,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,Make reservation on weekend,2013-01-27,0,CxDOIDnH8gp9KXzpBHJYXw,2022-08-12
2,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,Great place for couple has $7.99 dish,2013-01-27,0,CxDOIDnH8gp9KXzpBHJYXw,2022-08-12
3,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,King of bbq pork for $22,2013-01-27,0,CxDOIDnH8gp9KXzpBHJYXw,2022-08-12
4,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,Their lunch combos for small groups is a decen...,2013-01-29,0,Tc3GAQdAfOW542ROdyCZPg,2022-08-12
5,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,Make sure to request the delicious house soup....,2015-01-04,0,mFwRTTDW0Yr-rFkTF2cFsw,2022-08-12
6,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,"$7.50 lunch special, dish of rice\/noodles wit...",2017-01-15,0,0cUzu82KJiE5_xZA0Iu3ZQ,2022-08-12
7,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,$5 lunch special,2014-07-11,0,2oMkzQcRL7-d7URt3Xo_Xg,2022-08-12
8,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,$6 lunch special. A lot of selection on the lu...,2015-02-19,0,3yMtpQ_wV4ZGg6E69uE1PQ,2022-08-12
9,--6MefnULPED_I942VcFNA,--6MefnULPED_I942VcFNA,BBQ pork is sold out early on Saturday,2013-03-23,0,EiP1OFgs-XGcKZux0OKWIA,2022-08-12


In [17]:
# Run RestaurantAttributes ETL
etl_restaurant_attributes(business_df)

# Validate success
verify_inserts('RestaurantAttributes')

Reducing columns like BusinessParking
Reducing columns like Music
Reducing columns like Restaurants
Reducing columns like GoodForMeal
Reducing columns like DietaryRestrictions
Reducing columns like BestNights
Encoding column Alcohol
Encoding column NoiseLevel
Encoding column WiFi
Encoding column Smoking
Encoding column BYOBCorkage
business_id TEXT
change_date DATE
ByAppointmentOnly BOOLEAN
BusinessAcceptsCreditCards BOOLEAN
GoodForKids BOOLEAN
WheelchairAccessible BOOLEAN
BikeParking BOOLEAN
Alcohol TEXT
HasTV BOOLEAN
NoiseLevel TEXT
Caters BOOLEAN
WiFi TEXT
HappyHour BOOLEAN
GoodForDancing BOOLEAN
OutdoorSeating BOOLEAN
CoatCheck BOOLEAN
Smoking TEXT
DriveThru BOOLEAN
BYOBCorkage TEXT
BusinessParkingTypeId INTEGER
BusinessParkingType TEXT
MusicTypeId INTEGER
MusicType TEXT
RestaurantsTypeId INTEGER
RestaurantsType TEXT
GoodForMealTypeId INTEGER
GoodForMealType TEXT
DietaryRestrictionsTypeId INTEGER
DietaryRestrictionsType TEXT
BestNightsTypeId INTEGER
BestNightsType TEXT
AlcoholId INT

,business_id,change_date,ByAppointmentOnly,BusinessAcceptsCreditCards,GoodForKids,WheelchairAccessible,BikeParking,Alcohol,HasTV,NoiseLevel,...,GoodForMealType,DietaryRestrictionsTypeId,DietaryRestrictionsType,BestNightsTypeId,BestNightsType,AlcoholId,NoiseLevelId,WiFiId,SmokingId,BYOBCorkageId
0,XOSRcvtaKc_Q5H1SAzN20A,2022-08-12,False,False,False,True,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
1,l09JfMeQ6ynYs5MCJtrcmQ,2022-08-12,False,False,False,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
2,gAy4LYpsScrj8POnCW6btQ,2022-08-12,False,False,False,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
3,1_3nOM7s9WqnJWTNu2-i8Q,2022-08-12,False,False,True,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
4,BnuzcebyB1AfxH0kjNWqSg,2022-08-12,False,False,False,True,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
5,EJFdWX908N8Yc2XG0Lky8A,2022-08-12,False,False,False,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
6,F0fEKpTk7gAmuSFI0KW1eQ,2022-08-12,False,False,False,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
7,HAX1zec191t7QkT2sBZ76A,2022-08-12,False,False,False,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
8,1nhf9BPXOBFBkbRkpsFaxA,2022-08-12,False,False,False,True,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
9,T5CdfrZWw-uW9Y5L_sddqQ,2022-08-12,False,False,True,False,False,no,False,no,...,na,0,na,0,na,1,1,1,1,1
